In [ ]:
import cv2
from keras.models import load_model
import numpy as np
import time
import random

model = load_model('keras_model.h5')
cap = cv2.VideoCapture(0)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
started = False
next_round = True
countdown = False
counter = 0
elapsed = 0
round_time = 0
message = ''

def get_prediction_string(prediction):
    result = None
    if (prediction[0][0])>0.9:
        result = "Paper"
    elif prediction [0][1]>0.9:
        result = 'Rock'
    elif prediction [0][2]>0.9:
        result = 'Scissors'
    return result

while True: 
    ret, frame = cap.read()
    resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
    image_np = np.array(resized_frame)
    normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the imagekaras_model
    data[0] = normalized_image
    prediction = model.predict(data)
    print(prediction)
    if not started:
        message = 'Press a to strat'
    if cv2.waitKey(33) == ord ('a'):
        if not started:
            counter = time.time()
            started = True
            countdown = True
    if started:
        elapsed = 5 - (time.time() - counter)
        if elapsed <= -4:
            message = 'Press n to play the next round' 
            if cv2.waitKey(33) == ord ('n'):
                started = False
                elapsed = 0
        elif elapsed <= 0:
            countdown = False
            #prediction ='Rock'
            prediction_string = get_prediction_string(prediction)
            computer =  random.choice (['Rock','Paper','Scissors']) 
            message = f'You Showed {prediction_string}, and the computer showed {computer}.'
            if prediction_string == 'Rock':
                if computer == 'Rock':
                    message += "It's a draw"
                elif computer == 'Paper':
                    message += 'Sorry you lose'
                else: 
                    message += 'Great , you WIN!'
            elif prediction_string == 'Paper':
                if computer == 'Rock':
                    message += "Great , you WIN!"
                elif computer == 'Paper':
                    message += "It's a draw"
                else: 
                    message += 'Sorry you lose'
            elif prediction_string == 'Scissors':  
                if computer == 'Rock':
                    message += 'Sorry you lose'
                elif computer == 'Paper':
                    message += 'Great , you WIN!'
                else: 
                    message += "It's a draw"
            else:
                message += 'Wrong prediction'          
        if countdown:
            message = f' Show yuor hand in  {int (elapsed)} seconds'

    cv2.putText(frame, message, (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255), 1)
    cv2.imshow('frame', frame)
    #press q to close the window
    if cv2.waitKey(1) & 0xFF == ord ('q'):
       break            

cv2.waitKey (1)
# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()
